In [1]:
#import tensorflow as tf
#!pip install tensorflow==2.9.1
import tensorflow as tf
print(tf.__version__)
#!pip install mne
#from mne.decoding import CSP

2.9.1


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from IPython.display import clear_output
#import pywt
import numpy as np
#import imageio
import tensorflow as tf
#import cv2
#import time
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
from tensorflow.keras.models import Model, load_model
from sklearn.metrics import classification_report
import os
#from scipy import signal
#from sklearn import preprocessing
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
train_data = pd.read_csv('/content/drive/MyDrive/TCC-Physionet/PhysionetTrain.csv', delimiter=";")
test_data = pd.read_csv('/content/drive/MyDrive/TCC-Physionet/PhysionetTest.csv', delimiter=";")
val_data = pd.read_csv('/content/drive/MyDrive/TCC-Physionet/PhysionetVal.csv', delimiter=";")


In [5]:
labels = {'0': 'rest', '1': 'left', '2': 'right'}
selected_eletrodes = list(train_data['eletrodo'].unique())
 #['F3', 'F4', 'F5', 'F6', 'FC5', 'FC3','FC4', 'FC6', 'C5', 'C3', 'C4', 'C6']

In [6]:
# y_delta = butter_bandpass_filter(signal, 0.1, 4, 160, order=5)
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = signal.lfilter(b, a, data)
    return y

def metrics(data, selected_eletrodes):
  std = []
  mean = []
  for eletrode in selected_eletrodes:
    eletrodoStd = pd.Series(data.loc[data["eletrodo"] == eletrode, "leitura0":"leitura639"].values.ravel('F')).std()
    eletrodoMean = pd.Series(data.loc[data["eletrodo"] == eletrode, "leitura0":"leitura639"].values.ravel('F')).mean()
    std.append(eletrodoStd)
    mean.append(eletrodoMean)

  return std, mean

In [7]:
def assemble_data(data, selected_eletrodes, std_el, mean_el):
  cwt_limit = 30
  widths = np.arange(1, cwt_limit + 1)
  labels = []
  signals = []
  pacients = data["paciente"].unique()
  len_selected_eletrodes = len(selected_eletrodes)

  for pacient in pacients:
      measures = data.loc[data['paciente'] == pacient]['medicao'].unique()
      for measure in measures:
          label = data.loc[(data['paciente'] == pacient) & (data['medicao'] == measure)]['label'].unique()[0]
          signal_list = []
          img = np.zeros([len_selected_eletrodes, 640])
          print(pacient,measure)
          for idx, eletrode in enumerate(selected_eletrodes):
              sign = data.loc[(data['paciente'] == pacient) & (data['medicao'] == measure) & (data['eletrodo'] == eletrode)].iloc[:,4:644].to_numpy().reshape(640)
              sign -= mean_el[idx]
              sign /= std_el[idx]
              img[idx] = sign
          signals.append(img)
          labels.append(label-1)
          '''if label == 1 :
            labels.append([1,0,0])
          elif label == 2:
            labels.append([0,1,0])
          else:
            labels.append([0,0,1])'''
          clear_output(wait=True)

  return labels, signals


In [8]:
class PhysionetDataset:
  def __init__(self, data, selected_eletrodes, batch_size):
    self.std, self.mean = metrics(data, selected_eletrodes)
    label, signal = assemble_data(data, selected_eletrodes, self.std, self.mean)
    img_generator = tf.keras.preprocessing.image.ImageDataGenerator()

    self.features = img_generator.flow(
        x=np.array(signal).reshape(len(signal), len(selected_eletrodes), 640, 1),
        y=np.array(label).reshape(len(label),1),
        batch_size=batch_size)



In [9]:
batch = 64


train_dataset = PhysionetDataset(train_data, selected_eletrodes, batch)
val_dataset = PhysionetDataset(val_data, selected_eletrodes, batch)
#test_dataset = PhysionetDataset(test_data, selected_eletrodes, batch)

109 20


In [10]:
inp = layers.Input(shape=(64, 640, 1))
x = layers.Conv2D(128, (2,2), activation='relu')(inp)
x = layers.Conv2D(256, (2,2), activation='relu')(x)

x = layers.MaxPool2D((2,2))(x)

x = layers.Conv2D(512, (2,2), activation='relu')(x)
x = layers.MaxPool2D((4,4))(x)
x = layers.Conv2D(512, (2,2), activation='relu')(x)
x = layers.MaxPool2D((4,4))(x)


x = layers.TimeDistributed(layers.LSTM(256, return_sequences=True))(x)
x = layers.TimeDistributed(layers.LSTM(128, return_sequences=True))(x)
x = layers.TimeDistributed(layers.LSTM(64, return_sequences=True))(x)
x = layers.Flatten()(x)
#x = layers.Dropout(0.5)(x)
x = layers.Dense(3, activation='softmax')(x)

model = Model(inp, x)

In [11]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 640, 1)]      0         
                                                                 
 conv2d (Conv2D)             (None, 63, 639, 128)      640       
                                                                 
 conv2d_1 (Conv2D)           (None, 62, 638, 256)      131328    
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 319, 256)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 30, 318, 512)      524800    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 79, 512)       0         
 2D)                                                         

In [12]:

model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

In [13]:
#model = load_model('/content/drive/MyDrive/TCC-Physionet/model_CNNLSTM.h5')

history = model.fit(train_dataset.features.x, train_dataset.features.y, epochs=10,
                    validation_data=(val_dataset.features.x, val_dataset.features.y))#, callbacks=callbacks)

model.save('/content/drive/MyDrive/TCC-Physionet/model_CNNLSTM.h5')

Epoch 1/10
55/55 [==============================] - 681s 12s/step - loss: 1.1017 - accuracy: 0.3180 - val_loss: 1.0970 - val_accuracy: 0.3529
Epoch 2/10
55/55 [==============================] - 673s 12s/step - loss: 1.0991 - accuracy: 0.3720 - val_loss: 1.0815 - val_accuracy: 0.3941
Epoch 3/10
55/55 [==============================] - 671s 12s/step - loss: 1.0747 - accuracy: 0.4139 - val_loss: 1.0757 - val_accuracy: 0.4176
Epoch 4/10
55/55 [==============================] - 672s 12s/step - loss: 1.0294 - accuracy: 0.4724 - val_loss: 1.0371 - val_accuracy: 0.4824
Epoch 5/10
55/55 [==============================] - 681s 12s/step - loss: 0.9843 - accuracy: 0.5218 - val_loss: 1.0138 - val_accuracy: 0.5029
Epoch 6/10
55/55 [==============================] - 684s 12s/step - loss: 0.9200 - accuracy: 0.5666 - val_loss: 0.9378 - val_accuracy: 0.5794
Epoch 7/10
55/55 [==============================] - 678s 12s/step - loss: 0.8608 - accuracy: 0.5976 - val_loss: 0.8899 - val_accuracy: 0.5794
Epoch 

In [14]:
model = load_model('/content/drive/MyDrive/TCC-Physionet/model_CNNLSTM.h5')

history = model.fit(train_dataset.features.x, train_dataset.features.y, epochs=10,
                    validation_data=(val_dataset.features.x, val_dataset.features.y))#, callbacks=callbacks)

model.save('/content/drive/MyDrive/TCC-Physionet/model_CNNLSTM.h5')

Epoch 1/10
55/55 [==============================] - 675s 12s/step - loss: 0.6945 - accuracy: 0.7107 - val_loss: 0.9006 - val_accuracy: 0.5765
Epoch 2/10
55/55 [==============================] - 670s 12s/step - loss: 0.6274 - accuracy: 0.7423 - val_loss: 0.9730 - val_accuracy: 0.6000
Epoch 3/10
55/55 [==============================] - 670s 12s/step - loss: 0.5962 - accuracy: 0.7457 - val_loss: 0.9160 - val_accuracy: 0.6324
Epoch 4/10
55/55 [==============================] - 672s 12s/step - loss: 0.5200 - accuracy: 0.7870 - val_loss: 1.0189 - val_accuracy: 0.5971
Epoch 5/10
55/55 [==============================] - 676s 12s/step - loss: 0.4830 - accuracy: 0.7991 - val_loss: 0.9751 - val_accuracy: 0.6176
Epoch 6/10
55/55 [==============================] - 684s 12s/step - loss: 0.4313 - accuracy: 0.8318 - val_loss: 1.0378 - val_accuracy: 0.6500
Epoch 7/10
55/55 [==============================] - 686s 12s/step - loss: 0.3299 - accuracy: 0.8800 - val_loss: 1.2223 - val_accuracy: 0.6059
Epoch 

In [15]:
model = load_model('/content/drive/MyDrive/TCC-Physionet/model_CNNLSTM.h5')

history = model.fit(train_dataset.features.x, train_dataset.features.y, epochs=10,
                    validation_data=(val_dataset.features.x, val_dataset.features.y))#, callbacks=callbacks)

model.save('/content/drive/MyDrive/TCC-Physionet/model_CNNLSTM.h5')

Epoch 1/10
55/55 [==============================] - 704s 13s/step - loss: 0.1543 - accuracy: 0.9460 - val_loss: 1.6099 - val_accuracy: 0.6265
Epoch 2/10
55/55 [==============================] - 700s 13s/step - loss: 0.1012 - accuracy: 0.9644 - val_loss: 1.8762 - val_accuracy: 0.6206
Epoch 3/10
55/55 [==============================] - 699s 13s/step - loss: 0.0423 - accuracy: 0.9874 - val_loss: 2.0217 - val_accuracy: 0.6294
Epoch 4/10
55/55 [==============================] - 691s 13s/step - loss: 0.0269 - accuracy: 0.9925 - val_loss: 2.0653 - val_accuracy: 0.6118
Epoch 5/10
55/55 [==============================] - 691s 13s/step - loss: 0.0827 - accuracy: 0.9719 - val_loss: 2.1595 - val_accuracy: 0.5971
Epoch 6/10
55/55 [==============================] - 691s 13s/step - loss: 0.1328 - accuracy: 0.9483 - val_loss: 1.8843 - val_accuracy: 0.5941
Epoch 7/10
55/55 [==============================] - 695s 13s/step - loss: 0.0666 - accuracy: 0.9765 - val_loss: 1.7209 - val_accuracy: 0.6412
Epoch 

In [16]:
test_dataset = PhysionetDataset(test_data, selected_eletrodes, batch)

109 25


In [17]:
score = model.evaluate(test_dataset.features.x, test_dataset.features.y, verbose=0)
print('Loss: ', score[0])
print('Accuracy: ', score[1])

Loss:  2.0349764823913574
Accuracy:  0.594427227973938


In [ ]:
inp = layers.Input(shape=(64, 800, 1))
x = layers.Conv2D(256, (2,2), activation='relu')(inp)
x = layers.Conv2D(256, (2,2), activation='relu')(x)

x = layers.MaxPool2D((2,2))(x)

x = layers.Conv2D(512, (2,2), activation='relu')(x)
x = layers.Conv2D(512, (2,2), activation='relu')(x)

x = layers.MaxPool2D((2,2))(x)

x = layers.TimeDistributed(layers.LSTM(256, return_sequences=True))(x)
x = layers.TimeDistributed(layers.LSTM(128, return_sequences=True))(x)
x = layers.TimeDistributed(layers.LSTM(64, return_sequences=True))(x)
x = layers.Flatten()(x)
#x = layers.Dropout(0.5)(x)
x = layers.Dense(3, activation='softmax')(x)

model = Model(inp, x)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score

from mne import Epochs, pick_types, events_from_annotations
from mne.channels import make_standard_montage
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne.decoding import CSP
print(__doc__)


Automatically created module for IPython interactive environment


In [ ]:


# #############################################################################
# # Set parameters and read data

# avoid classification of evoked responses by using epochs that start 1s after
# cue onset.
tmin, tmax = -1.0, 4.0
event_id = dict(rest=1,left=2, right=3)
subject = 1
runs = [4]

raw_fnames = eegbci.load_data(subject, runs)
raw = concatenate_raws([read_raw_edf(f, preload=True) for f in raw_fnames])
eegbci.standardize(raw)  # set channel names
montage = make_standard_montage("standard_1005")
raw.set_montage(montage)


Download complete in 03s (2.5 MB)
Extracting EDF parameters from /root/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...


<RawEDF | S001R04.edf, 64 x 20000 (125.0 s), ~9.9 MB, data loaded>

In [ ]:
# Apply band-pass filter
#raw.filter(2.0, 10.0, fir_design="firwin", skip_by_annotation="edge")

events, _ = events_from_annotations(raw, event_id='auto')
print(events)
picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False, exclude="bads")


Used Annotations descriptions: ['T0', 'T1', 'T2']
[[    0     0     1]
 [  672     0     3]
 [ 1328     0     1]
 [ 2000     0     2]
 [ 2656     0     1]
 [ 3328     0     2]
 [ 3984     0     1]
 [ 4656     0     3]
 [ 5312     0     1]
 [ 5984     0     3]
 [ 6640     0     1]
 [ 7312     0     2]
 [ 7968     0     1]
 [ 8640     0     3]
 [ 9296     0     1]
 [ 9968     0     2]
 [10624     0     1]
 [11296     0     3]
 [11952     0     1]
 [12624     0     2]
 [13280     0     1]
 [13952     0     2]
 [14608     0     1]
 [15280     0     3]
 [15936     0     1]
 [16608     0     2]
 [17264     0     1]
 [17936     0     3]
 [18592     0     1]
 [19264     0     2]]


In [ ]:
# Read epochs (train will be done only between 1 and 2s)
# Testing will be done with a running classifier
epochs = Epochs(
    raw,
    events,
    event_id,
    tmin,
    tmax,
    proj=True,
    picks=picks,
    baseline=None,
    preload=True,
)
epochs_train = epochs.copy().crop(tmin=1.0, tmax=2.0)
labels = epochs.events[:, -1] - 2

Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 30 events and 801 original time points ...
1 bad epochs dropped


In [ ]:
# Define a monte-carlo cross-validation generator (reduce variance):
scores = []
epochs_data = epochs.get_data()
epochs_data_train = epochs_train.get_data()
cv = ShuffleSplit(10, test_size=0.2, random_state=42)
cv_split = cv.split(epochs_data_train)

# Assemble a classifier
lda = LinearDiscriminantAnalysis()
csp = CSP(n_components=12, reg=None, log=True, norm_trace=False)

# Use scikit-learn Pipeline with cross_val_score function
clf = Pipeline([("CSP", csp), ("LDA", lda)])
scores = cross_val_score(clf, epochs_data_train, labels, cv=cv, n_jobs=None)

# Printing the results
class_balance = np.mean(labels == labels[0])
class_balance = max(class_balance, 1.0 - class_balance)
print(
    "Classification accuracy: %f / Chance level: %f" % (np.mean(scores), class_balance)
)

# plot CSP patterns estimated on full data for visualization
retorno = csp.fit_transform(epochs_data, labels)

csp.plot_patterns(epochs.info, ch_type="eeg", units="Patterns (AU)", size=1.5)

In [ ]:
print(len(epochs_data),len(epochs_data[0]), epochs_data[0][0])
print(labels)

In [ ]:
print(len(retorno))
print(len(retorno),len(retorno[0]),retorno[0][0])

29
29 12 -0.593578956955287


In [ ]:
sfreq = raw.info["sfreq"]
w_length = int(sfreq * 0.5)  # running classifier: window length
w_step = int(sfreq * 0.1)  # running classifier: window step size
w_start = np.arange(0, epochs_data.shape[2] - w_length, w_step)

scores_windows = []

for train_idx, test_idx in cv_split:
    y_train, y_test = labels[train_idx], labels[test_idx]

    X_train = csp.fit_transform(epochs_data_train[train_idx], y_train)
    X_test = csp.transform(epochs_data_train[test_idx])

    # fit classifier
    lda.fit(X_train, y_train)

    # running classifier: test classifier on sliding window
    score_this_window = []
    for n in w_start:
        X_test = csp.transform(epochs_data[test_idx][:, :, n : (n + w_length)])
        score_this_window.append(lda.score(X_test, y_test))
    scores_windows.append(score_this_window)

# Plot scores over time
w_times = (w_start + w_length / 2.0) / sfreq + epochs.tmin

plt.figure()
plt.plot(w_times, np.mean(scores_windows, 0), label="Score")
plt.axvline(0, linestyle="--", color="k", label="Onset")
plt.axhline(0.5, linestyle="-", color="k", label="Chance")
plt.xlabel("time (s)")
plt.ylabel("classification accuracy")
plt.title("Classification score over time")
plt.legend(loc="lower right")
plt.show()